In [12]:
!pip install huggingface_hub
!pip install -q transformers
!pip install -q pytorch-lightning
!pip install -q wandb
!pip install -q roboflow
!pip install -q rembg

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 42.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.5/296.5 kB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 58.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.9/66.9 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
from transformers import AutoImageProcessor, AutoModelForObjectDetection
import torch
from PIL import Image
import requests
from io import BytesIO
import os


image_processor = AutoImageProcessor.from_pretrained("hustvl/yolos-tiny")
model = AutoModelForObjectDetection.from_pretrained("hustvl/yolos-tiny")

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.
The `max_size` parameter is deprecated and will be removed in v4.26. Please specify in `size['longest_edge'] instead`.


In [16]:
def crop_save_scenes(input_img_path, output_crops_path):
    img = Image.open(input_img_path)
    inputs = image_processor(images=img, return_tensors="pt")
    outputs = model(**inputs)
    idx = 0

    # convert outputs (bounding boxes and class logits) to COCO API
    target_sizes = torch.tensor([img.size[::-1]])
    results = image_processor.post_process_object_detection(outputs, threshold=0.9, target_sizes=target_sizes)[0]

    for score, label, box in zip(results["scores"], results["labels"], results["boxes"]):
        if model.config.id2label[label.item()] == 'person':
            idx += 1
            box = [round(i, 2) for i in box.tolist()]
            print(
                f"Detected {model.config.id2label[label.item()]} with confidence "
                f"{round(score.item(), 3)} at location {box}"
            )
            img_to_save = img.crop((box[0], box[1], box[2], box[3])) 

            ### REPLACE THIS WITH CODE TO SAVE THE CROPPED IMAGES LOCALLY, NOT IN GOOGLE DRIVE
            img_to_save.save(f"{output_crops_path}/{idx}.jpeg")

In [19]:
crop_save_scenes("street.jpeg", "/content/example_street")

Detected person with confidence 0.999 at location [2222.8, 18.4, 2669.08, 1349.34]
Detected person with confidence 0.992 at location [1121.34, 76.75, 1768.77, 1667.35]
Detected person with confidence 0.999 at location [725.04, 17.69, 1182.74, 1353.88]
Detected person with confidence 0.994 at location [0.72, -1.62, 449.35, 1504.58]
Detected person with confidence 0.991 at location [2546.01, 2.71, 3000.04, 1674.46]


In [20]:
from pathlib import Path
from rembg import remove, new_session

def remove_background(path_to_parent_folder):
    session = new_session()

    for file in Path(path_to_parent_folder).glob('*.jpeg'):
        input_path = str(file)
        output_path = str(file.parent / (file.stem + ".out.jpeg"))

        with open(input_path, 'rb') as i:
            with open(output_path, 'wb') as o:
                input = i.read()
                output = remove(input, session=session)
                o.write(output)

100%|███████████████████████████████████████| 176M/176M [00:00<00:00, 48.4GB/s]


In [ ]:
remove_background("/content/example_street")